In [1]:
import numpy as np
import heapq

In [2]:
def get_distance(p1, p2):
  return np.sqrt(np.sum(np.square(p1 - p2)))

OBS: heapsort ( - memória auxiliar)

In [29]:
class Node:
  def __init__(self, left_node, right_node, point):
    self.left_node = left_node
    self.right_node = right_node
    self.point = point

  def is_leaf(self):
    return self.left_node == None and self.right_node == None

class Kd_tree:
  def __init__(self, dataset, ndim):
    self.ndim = ndim
    self.root = self.build_kd_tree(dataset)

  def build_kd_tree(self, dataset, depth=0):
    dataset_size = len(dataset)
    current_dimension = (depth) % self.ndim
    dataset = dataset[np.argsort(dataset[:, current_dimension], kind='heapsort')]
    if(dataset_size) == 1:
      return Node(None, None, dataset[0])
    else:
      half = dataset_size // 2
      left = self.build_kd_tree(dataset[: half], depth+1)
      right = self.build_kd_tree(dataset[half :], depth+1)
      return Node(left, right, dataset[half-1]) 

  def recursive_k_nearest(self, k, current_node, point, priority_heap, depth=0):
    if(current_node == None):
      return
    current_dimension = depth % self.ndim
    distance = get_distance(point, current_node.point)
  
    if(current_node.is_leaf() and len(priority_heap) < k):
      heapq.heappush(priority_heap, (-distance, current_node.point))
    elif(current_node.is_leaf() and distance < -priority_heap[0][0]):
      heapq.heappushpop(priority_heap, (-distance, current_node.point))

    next_branch, other_branch = None, None

    if(point[current_dimension] < current_node.point[current_dimension]):
      next_branch, other_branch = current_node.left_node, current_node.right_node
    else: 
      next_branch, other_branch = current_node.right_node, current_node.left_node

    self.recursive_k_nearest(k, next_branch, point, priority_heap, depth+1)
    line_shortest_distance = point[current_dimension] - current_node.point[current_dimension]

    if(distance >= line_shortest_distance ** 2 or len(priority_heap) < k ):
      self.recursive_k_nearest(k, other_branch, point, priority_heap, depth+1)

  def get_k_nearest(self, k, point):
    heap = []
    self.recursive_k_nearest(k, self.root, point, heap)
    return heap

  def get_three_structure(self, current_node, depth = 0):
    if(current_node.right_node == None and current_node.left_node == None):
      print(f'{depth}: {current_node.point}')
    else:
      self.get_three_structure(current_node.left_node, depth+1)
      self.get_three_structure(current_node.right_node, depth+1)

  def print_three(self):
    self.get_three_structure(self.root)

In [16]:
data = np.array([
    [4,4,1],
    [3,3,1],
    [2,2,1],
    [1,1,1],
    [5,5,1],
    [6,6,1],
    [8,8,1],
    [7,7,1],
])

tree = Kd_tree(data, 2)
tree.get_k_nearest(4, np.array([0,0,1]))


[(-5.656854249492381, array([4, 4, 1])),
 (-4.242640687119285, array([3, 3, 1])),
 (-2.8284271247461903, array([2, 2, 1])),
 (-1.4142135623730951, array([1, 1, 1]))]

In [18]:

def get_datasets_from_file(filepath, train_size = 0.7, seed = 42):
  dataset = np.genfromtxt(filepath, delimiter = ',')
  num_train_instances = int(len(dataset) * train_size)
  if(seed is not None):
    np.random.seed(seed)
  np.random.shuffle(dataset)
  train_set, test_set = dataset[:num_train_instances], dataset[num_train_instances:]
  return (train_set, test_set)

In [20]:
train_set, test_set = get_datasets_from_file('banana.csv')

In [30]:
class K_neighbours_classifier:
  def __init__(self, k = 2):
    self.k = k
  
  def fit(self, dataset, num_of_features):
    self.kd_tree = Kd_tree(dataset, num_of_features)

  def predict(self, point):
    knn_heap = self.kd_tree.get_k_nearest(self.k, np.array(point))
    